In [1]:
import os

curr_folder = os.getcwd() + "/"
datasets_folder = curr_folder[:-6] + "Datasets/"
datasets_folder

'/home/ipsilva/Repositories/dynamic-projections/Datasets/'

In [2]:
datasets_subfolders = []
for i in os.walk(datasets_folder):
    datasets_subfolders.append(i[0])

datasets_subfolders = datasets_subfolders[1:]
datasets_subfolders

['/home/ipsilva/Repositories/dynamic-projections/Datasets/gaussians',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/quickdraw',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/sorts',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/walk',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/nnset',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/fashion',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cifar10cnn',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/esc50',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/qtables',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cartolastd']

In [3]:
dataset_number = 8
selected_dataset = datasets_subfolders[dataset_number]
selected_dataset

'/home/ipsilva/Repositories/dynamic-projections/Datasets/qtables'

In [4]:
from os import listdir
from os.path import isfile, join

files = [f for f in listdir(selected_dataset) if isfile(join(selected_dataset, f))]
files[:5]

['qtables-32.csv',
 'qtables-12.csv',
 'qtables-29.csv',
 'qtables-36.csv',
 'qtables-15.csv']

In [5]:
from sklearn.preprocessing import MinMaxScaler
from itertools import product
import pandas as pd
import numpy as np

df = pd.DataFrame()

obs_per_timestep = []
for file in files:
    file_loc = selected_dataset + "/" + file
    ndf = pd.read_csv(file_loc)
    df = df.append(ndf)
    obs_per_timestep.append(ndf.shape[0])
    
median_obs_per_timestep = np.median(obs_per_timestep)
dev_obs_per_timestep = np.std(obs_per_timestep)

min_max_scaler = MinMaxScaler()
df = pd.DataFrame(min_max_scaler.fit_transform(df.drop(columns=df.columns[0], axis=1)))
print(df.shape, df.head())

(7200, 1200)        0         1         2         3         4         5         6     \
0  0.244097  0.960222  0.861622  0.458094  0.958434  0.414613  0.913781   
1  0.790785  0.547854  0.603857  0.416086  0.454480  0.579186  0.956116   
2  0.549191  0.827718  0.863565  0.671931  0.868700  0.205783  0.965113   
3  0.832012  0.006982  0.443634  0.600459  0.521659  0.806105  0.983887   
4  0.255539  0.783254  0.161052  0.767967  0.367936  0.247120  0.911428   

       7         8         9     ...      1190      1191      1192      1193  \
0  0.900889  0.932073  0.933234  ...  0.853521  0.189018  0.824095  0.821336   
1  0.967704  0.925711  0.834022  ...  0.504708  0.313264  0.980895  0.124184   
2  0.958909  0.920791  0.871993  ...  0.357546  0.503435  0.657478  0.622299   
3  0.912745  0.923175  0.933729  ...  0.183455  0.660797  0.154280  0.566164   
4  0.976703  0.994208  0.959203  ...  0.369520  0.528775  0.756597  0.302453   

       1194      1195      1196      1197      1198    

In [6]:
from sklearn.decomposition import PCA

def intrisic_dim(dataframe, target_variance):
    pca = PCA()

    principalComponents = pca.fit_transform(df)

    list_variance = pca.explained_variance_ratio_

    final_var = 0
    dims = 0
    for var in list_variance:
        final_var += var
        dims += 1
        if final_var > target_variance:
            break
    return dims, final_var, principalComponents[:dims]

In [7]:
target_variance = 0.9

(dims, final_var, components) = intrisic_dim(df, target_variance)

print("Total time-steps:", len(obs_per_timestep))
print("Registers per time-step (median):", median_obs_per_timestep)
print("Registers per time-step (deviation):", dev_obs_per_timestep)
print("Total dimensions:", df.shape[1])
print("Intrinsic dimensions:", dims)
print("Total variance:", final_var)

Total time-steps: 40
Registers per time-step (median): 180.0
Registers per time-step (deviation): 0.0
Total dimensions: 1200
Intrinsic dimensions: 79
Total variance: 0.9012071053796678
